# Task 2: Fine-Grained Classification

## Instructions on How to run the model

To execute the model, ensure you follow these steps:

1. Mount a Google drive to a specified path.
2. **Import all required libraries** and then later load the dataset using `section 2`.
3. Run the functions within Image Preprocessing and Feature Engineering.
4. We can **skip the Model building process** as I've already trained the model.
5. Run the `evaluation function` to use it later during testing the model.
6. No need to retrain the model by executing `Section 7`.
7. If `test.csv` file contains filenames and images are stored in the designated directory, then you can process with the `Model Testing` section.

___

## 0. Mounting a drive

We'll first mount the drive to the directory '/content/drive'. Afterward, we'll switch the working directory to the folder containing this file.

In [1]:
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Deep Learning/Coursework/Task 2")

Mounted at /content/drive


## 1. Import Required Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.data.experimental import AUTOTUNE
from tensorflow.io import read_file
from tensorflow.image import decode_jpeg, resize, random_flip_left_right, crop_to_bounding_box


# For AUTOTUNE
AUTO = AUTOTUNE

SEED_VALUE = 125
np.random.seed(SEED_VALUE)
tf.compat.v1.set_random_seed(SEED_VALUE)

ROOT_FOLDER = "/content/drive/MyDrive/Deep Learning/Coursework/Task 2"

## 2. Load Data

1. The images are store inside directory `./classification_aircraft/fgvc-aircraft-2013b/data/images`.
2. I've got three different csv files for training, validation, testing. Later in the code, I stored full path of the images (training, validation, testing) in `train_paths`, `val_paths`, `test_paths`.
3. The labels are undergoing conversion to categorical format.



In [3]:
train_data = pd.read_csv(f'{ROOT_FOLDER}/classification_aircraft/train.csv')
test_data = pd.read_csv(f'{ROOT_FOLDER}/classification_aircraft/test.csv')
val_data = pd.read_csv(f'{ROOT_FOLDER}/classification_aircraft/val.csv')

IMG_PATH = f'{ROOT_FOLDER}/classification_aircraft/fgvc-aircraft-2013b/data/images'
train_paths = train_data.filename.apply(lambda x: os.path.join(IMG_PATH, x))
train_labels = to_categorical(train_data.Labels)

val_paths = val_data.filename.apply(lambda x: os.path.join(IMG_PATH, x))
val_labels = to_categorical(val_data.Labels)

test_paths = test_data.filename.apply(lambda x: os.path.join(IMG_PATH, x))
test_labels = to_categorical(test_data.Labels)


## 3. Image Preprocessing

1. In the below function, `decode_jpeg` is from TensorFlow's image module used to decode JPEG-encoded images into a format that TensorFlow can manipulate and further process or analyzed using TensorFlow operations
2. Later in the function, the image resized to given image_size (i.e. (521, 521)).

In [4]:
def image_decoding(filename, label=None, image_size=(512, 512)):
    bits = read_file(filename)
    img = decode_jpeg(bits, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    img = resize(img, image_size)

    if label is None:
        return img
    else:
        return img, label


## 4. Feature Engineering

1. The function `random_flip_left_right` randomly flips images from left to right. This operation is specifically used for data augmentation.
2. `crop_to_bounding_box` function crops the image in specified bounding box region. **In our images, there are noise from 20px bottom to top.**
3. The above function are mapped to all the images using TensorFlow operation.

In [5]:
def data_augmentation(image, label=None, image_size=(512, 512)):
    img = random_flip_left_right(image)
    img = crop_to_bounding_box(img, 0, 0, 512-20, 512)
    img = resize(img, image_size)

    if label is None:
        return img
    else:
        return img, label

In [6]:
batch_size = 32

train_dataset = (
    tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
    .map(image_decoding, num_parallel_calls=AUTO).map(data_augmentation, num_parallel_calls=AUTO).shuffle(2048).batch(batch_size)
)

val_dataset = (
    tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
    .map(image_decoding, num_parallel_calls=AUTO).batch(batch_size)
)

test_dataset = (tf.data.Dataset.from_tensor_slices(test_paths)
    .map(image_decoding, num_parallel_calls=AUTO).batch(batch_size)
)

## 5. Model Building

1. In this model, I've used DenseNet201 pre-trained model with `imagenet` weights.
2. Here, We are trying to train different dataset using `imagenet` weights, this concept is called **transfer learning**.
3. Chaning the architecture of the pre-trained model (eg. DenseNet201) is called **fine tuning.**
4. Last layer is output layer where activation function is `softmax`.
5. I've used Stochastic Gradient Descent (SGB) with learning rate 0.0005

In [ ]:
model = tf.keras.Sequential([
        DenseNet201(weights = 'imagenet',
              include_top = False,
              input_shape = (512, 512, 3),
              ),
        BatchNormalization(),
        Dropout(0.95),

        Flatten(),
        Dense(100, activation = 'softmax')
])

# The optimiser is stochastic gradient descent with a learning rate of 0.0005:
optimizer = SGD(learning_rate=0.0005)
model.compile(optimizer = optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

model.summary()

74836368/74836368 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 16, 16, 1920)      18321984  
                                                                 
 batch_normalization (Batch  (None, 16, 16, 1920)      7680      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 16, 16, 1920)      0         
                                                                 
 flatten (Flatten)           (None, 491520)            0         
                                                                 
 dense (Dense)               (None, 100)               49152100  
                                                                 
Total params: 67481764 (257.42 MB)
Trainable params: 6724

## 6. Evaluation

For the evaluation of the model, I've used a function which calculates `Accuracy, Precision, Recall, F1 Score, and Confusion Matrix`.

In [7]:
def evaluation(labels, predictions):
  # Calculate accuracy
  accuracy = round(accuracy_score(labels, predictions)*100, 3)

  # Calculate precision
  precision = round(precision_score(labels, predictions, average='macro')*100, 3)

  # Calculate recall
  recall = round(recall_score(labels, predictions, average='macro')*100, 3)

  # Calculate F1-score
  f1 = round(f1_score(labels, predictions, average='macro')*100, 3)

  # Calculate confusion matrix
  cm = confusion_matrix(labels, predictions)

  print(f"Accuracy: {accuracy}\n")
  print(f"Precision: {precision}\n")
  print(f"Recall: {recall}\n")
  print(f"F1-score: {f1}\n")
  print("Confusion Matrix:\n")
  print(cm)

## 7. Model Training

1. I've utilized a callback called `ReduceLROnPlateau` - It **adjusts learning rate dynamically!** ReduceLROnPlateau reduce models learning rate when specified matrix stops improving.

1. I've trained the model with `40` epochs and later the weights of the model are stored in the specified directory with name `aircraft_weights_checkpoints`.

In [ ]:
EPOCHS = 40
print('Training...\n')

reduce_lr = ReduceLROnPlateau(
              monitor = 'val_loss',
              factor = 0.5,
              patience = 5,
              min_lr = 0.0001)

history = model.fit(train_dataset,
                batch_size = batch_size,
                epochs = EPOCHS,
                validation_data = val_dataset,
                callbacks = [reduce_lr],
                verbose = 1)

# Save the model
model.save(f'{ROOT_FOLDER}/aircraft_weights_checkpoints', save_format='tf')
print("************* Model saved successfully *************")

Training...

Epoch 1/40
105/105 [==============================] - 629s 3s/step - loss: 10.4559 - accuracy: 0.0195 - val_loss: 3.9107 - val_accuracy: 0.1623 - lr: 5.0000e-04
Epoch 2/40
105/105 [==============================] - 87s 773ms/step - loss: 7.2402 - accuracy: 0.1437 - val_loss: 2.7138 - val_accuracy: 0.3315 - lr: 5.0000e-04
Epoch 3/40
105/105 [==============================] - 87s 774ms/step - loss: 5.1860 - accuracy: 0.2642 - val_loss: 2.4072 - val_accuracy: 0.4173 - lr: 5.0000e-04
Epoch 4/40
105/105 [==============================] - 87s 773ms/step - loss: 3.7119 - accuracy: 0.4091 - val_loss: 2.2177 - val_accuracy: 0.4773 - lr: 5.0000e-04
Epoch 5/40
105/105 [==============================] - 87s 773ms/step - loss: 3.0577 - accuracy: 0.4880 - val_loss: 2.1943 - val_accuracy: 0.4962 - lr: 5.0000e-04
Epoch 6/40
105/105 [==============================] - 87s 773ms/step - loss: 2.4289 - accuracy: 0.5621 - val_loss: 1.7474 - val_accuracy: 0.5590 - lr: 5.0000e-04
Epoch 7/40
105/1

## 8. Model Testing

1. In this, firstly the model is loaded from the directory, followed by prediction process taken place. The prediction was performed on the testing dataset.
2. The `Accuracy` was more than *70.057%*. The `Precision` was *71.418%*, The `Recall` was *70.058%* and I've got an `F1 Score` of *69.829%*.
3. Lastly, the `Confusion Matrix` was calculated for the testing dataset.

In [8]:
model = load_model(f'{ROOT_FOLDER}/aircraft_weights_checkpoints')
pred = model.predict(test_dataset, verbose=1)
test_data['Prediction'] = np.argmax(pred, axis=-1)

labels = test_data['Labels']
predictions = test_data['Prediction']
evaluation(labels, predictions)

105/105 [==============================] - 311s 2s/step
Accuracy: 70.057

Precision: 71.418

Recall: 70.058

F1-score: 69.829

Confusion Matrix:

[[20  0  0 ...  0  0  0]
 [ 1 26  0 ...  0  1  0]
 [ 0  0 14 ...  0  0  0]
 ...
 [ 0  0  0 ... 28  0  0]
 [ 0  1  0 ...  0 31  0]
 [ 0  0  0 ...  0  0 31]]


## 9. Demo

In [ ]:
model = load_model(f'{ROOT_FOLDER}/aircraft_weights_checkpoints')
testing_data = pd.read_csv(f'{ROOT_FOLDER}/classification_aircraft/test.csv')

IMG_PATH = f'{ROOT_FOLDER}/classification_aircraft/fgvc-aircraft-2013b/data/images'

testing_paths = testing_data.filename.apply(lambda x: os.path.join(IMG_PATH, x))
testing_labels = to_categorical(testing_data.Labels)

testing_dataset = (tf.data.Dataset.from_tensor_slices(testing_paths)
    .map(image_decoding, num_parallel_calls=AUTO).batch(batch_size)
)

pred = model.predict(testing_dataset, verbose=1)
testing_data['Prediction'] = np.argmax(pred, axis=-1)

labels = testing_data['Labels']
predictions = testing_data['Prediction']

# Evaluation
evaluation(labels, predictions)

105/105 [==============================] - 14s 138ms/step
Accuracy: 70.057

Precision: 71.418

Recall: 70.058

F1-score: 69.829

Confusion Matrix:

[[20  0  0 ...  0  0  0]
 [ 1 26  0 ...  0  1  0]
 [ 0  0 14 ...  0  0  0]
 ...
 [ 0  0  0 ... 28  0  0]
 [ 0  1  0 ...  0 31  0]
 [ 0  0  0 ...  0  0 31]]


___